<hr style="border:none;height:6px;background:#fff;margin:1em 0;">


<div style="text-align: center;">
  <h1>Empirical Asset Pricing with ML in Europe</h1>
  <h3>HEC Liege</h3>
  <h4><em>Lucas Dubois and Myriam Lamborelle</em></h4>
</div>

<hr style="border:none;height:6px;background:#fff;margin:1em 0;">


In [1991]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import matplotlib as mpl
from dateutil.relativedelta import relativedelta
from statsmodels.stats import diagnostic
from statsmodels.stats.sandwich_covariance import cov_hac
from statsmodels.stats.sandwich_covariance import se_cov
from stargazer.stargazer import Stargazer
from linearmodels.panel import PanelOLS
from linearmodels.panel import PooledOLS
import statsmodels.formula.api as smf
import warnings
import re

In [1992]:
path="/Users/lucasdubois/Desktop/LaTeX/EAP-ML/CODE/DATA/"

In [1993]:
figure_path="/Users/lucasdubois/Desktop/LaTeX/EAP-ML/CODE/Images/"

In [1994]:
offwhite = (230/255, 230/255, 220/255)
midnight = (0/255, 22/255, 36/255)
steelblue = (171/255, 193/255, 223/255)
primaryred = (127/255, 20/255, 22/255)
harmonizedblue =(48/255,88/255,140/255)

<hr style="border:none;height:4px;background:#fff;margin:1em 0;">


<div style="text-align: align;">
  <h2> <small>1</small>&nbsp;&nbsp;&nbsp;&nbsp;Database:</h2>
</div>

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">








<div style="text-align: align;">
  <h3> <small>1.1</small>&nbsp;&nbsp;&nbsp;&nbsp;Stocks:</h3>
</div>

In [1999]:
df = pd.read_excel(path + "stoxx600.xlsx")

df.rename(columns={df.columns[0]: "Date"}, inplace=True)
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

In [2000]:
df = df.loc[:, ~df.columns.str.startswith("#ERROR")]

In [2001]:
df_long = df.melt(
    id_vars="Date",
    var_name="col",
    value_name="value"
)

In [2002]:
df_long[["Stock", "Datatype"]] = (
    df_long["col"]
    .str.split(" - ", n=1, expand=True)
)

In [2003]:
df_panel = (
    df_long
    .pivot_table(
        index=["Date", "Stock"],
        columns="Datatype",
        values="value"
    )
    .reset_index()
)


In [2004]:
df_panel = df_panel.sort_values(["Stock", "Date"])

print(df_panel.head())
print(df_panel.tail())
print(df_panel.info())


Datatype       Date     Stock  DIVIDEND YIELD  MARKET VALUE  \
0        2005-01-01  3I GROUP            2.13       4085.77   
435      2005-02-01  3I GROUP            2.00       4355.70   
871      2005-03-01  3I GROUP            2.04       4279.01   
1307     2005-04-01  3I GROUP            2.11       4134.96   
1743     2005-05-01  3I GROUP            2.23       3913.99   

Datatype  MRKT VALUE TO BOOK   PER  PRICE INDEX  TOT RETURN IND  \
0                       1.12  30.4        244.9          302.56   
435                     1.20  32.4        261.0          322.55   
871                     1.18  31.8        256.4          316.87   
1307                    1.03  30.7        247.4          305.74   
1743                    0.98  29.1        234.6          289.84   

Datatype  TURNOVER BY VALUE  TURNOVER BY VOLUME  
0                  340749.3             83500.9  
435                338096.4             81321.8  
871                446679.1            100759.9  
1307              

In [2005]:
df_panel["Stock"].nunique()

599

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>1.2</small>&nbsp;&nbsp;&nbsp;&nbsp;Risk-Free:</h3>
</div>

In [2008]:
rf_df = pd.read_excel(path + "EURIBOR.xlsx", header=None)

In [2009]:
rf_df.columns = rf_df.iloc[0]
rf_df = rf_df.iloc[1:].reset_index(drop=True)


In [2010]:
rf_df.head()
rf_df.columns


Index(['DATE', 'TIME PERIOD', 'OBS.VALUE'], dtype='object', name=0)

In [2011]:
rf_df = rf_df.rename(columns={
    "DATE": "Date",
    "OBS.VALUE": "Rf"
})
rf_df = rf_df.drop(columns=["TIME PERIOD"])

In [2012]:
rf_df["Date"] = pd.to_datetime(rf_df["Date"])
rf_df["Rf"] = rf_df["Rf"].astype(float) / 100 / 12

In [2013]:
rf_df["Date"] = rf_df["Date"] + pd.offsets.MonthBegin(1)


In [2014]:
rf_df.head

<bound method NDFrame.head of 0         Date        Rf
0   2005-01-01  0.001806
1   2005-02-01  0.001759
2   2005-03-01  0.001753
3   2005-04-01  0.001753
4   2005-05-01  0.001754
..         ...       ...
247 2025-08-01  0.001577
248 2025-09-01  0.001575
249 2025-10-01  0.001581
250 2025-11-01  0.001589
251 2025-12-01  0.001588

[252 rows x 2 columns]>

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>1.3</small>&nbsp;&nbsp;&nbsp;&nbsp;Merging:</h3>
</div>

In [2017]:
df_almost = df_panel.merge(
    rf_df,
    on="Date",
    how="left"
)

In [2018]:
df_almost["Rf"].describe()

count    133070.000000
mean          0.000881
std           0.001363
min          -0.000497
25%          -0.000308
50%           0.000216
75%           0.001866
max           0.004026
Name: Rf, dtype: float64

In [2019]:
df_almost.to_csv(path + "Data1.csv", index=False)

<hr style="border:none;height:4px;background:#fff;margin:1em 0;">


<div style="text-align: align;">
  <h2> <small>2</small>&nbsp;&nbsp;&nbsp;&nbsp;Data Management:</h2>
</div>

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>2.1</small>&nbsp;&nbsp;&nbsp;&nbsp;From Long to Panel</h3>
</div>

In [2024]:
cols = df_almost.columns.tolist()
cols_no_date = [c for c in cols if c != "Date"]


In [2025]:
df_long = df_almost.melt(
    id_vars=["Date", "Stock"],
    var_name="Variable",
    value_name="value"
)

In [2026]:
df_long.columns


Index(['Date', 'Stock', 'Variable', 'value'], dtype='object')

In [2027]:
df_long.columns
df_long.head()

,Date,Stock,Variable,value
0,2005-01-01,3I GROUP,DIVIDEND YIELD,2.13
1,2005-02-01,3I GROUP,DIVIDEND YIELD,2.00
2,2005-03-01,3I GROUP,DIVIDEND YIELD,2.04
3,2005-04-01,3I GROUP,DIVIDEND YIELD,2.11
4,2005-05-01,3I GROUP,DIVIDEND YIELD,2.23


In [2028]:
df_panel = (
    df_long
    .pivot_table(
        index=["Date", "Stock"],
        columns="Variable",
        values="value"
    )
    .reset_index()
)

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>2.2.</small>&nbsp;&nbsp;&nbsp;&nbsp;Excess Returns:</h3>
</div>

In [2031]:
df_panel["RET"] = (
    df_panel
    .groupby("Stock")["TOT RETURN IND"]
    .pct_change(fill_method=None)
)

In [2032]:
df_panel["RET_FWD"] = df_panel.groupby("Stock")["RET"].shift(-1)
df_panel["EXCESS_RET_FWD"] = df_panel["RET_FWD"] - df_panel["Rf"]


<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>2.3</small>&nbsp;&nbsp;&nbsp;&nbsp;Firm Characteristics:</h3>
</div>

In [2035]:
def safe_log(x):
    out = np.full_like(x, np.nan, dtype="float64")
    mask = x > 0
    out[mask] = np.log(x[mask])
    return out

In [2036]:
df_panel["SIZE"] = safe_log(df_panel["MARKET VALUE"])
df_panel["BM"] = 1 / df_panel["MRKT VALUE TO BOOK"] 
df_panel["DY"] = df_panel["DIVIDEND YIELD"] / 100 #Dividend Yield was not in so we've changed it %
df_panel["EY"] = 1 / df_panel["PER"]
df_panel["MOM3"] = (
    df_panel
    .groupby("Stock")["PRICE INDEX"]
    .pct_change(2, fill_method=None)
)

df_panel["MOM12"] = (
    df_panel
    .groupby("Stock")["PRICE INDEX"]
    .pct_change(11, fill_method=None)
)

df_panel["VOL12"] = (
    df_panel
    .groupby("Stock")["RET"]
    .rolling(12)
    .std()
    .reset_index(level=0, drop=True)
)
df_panel["TURN_VAL"] = safe_log(df_panel["TURNOVER BY VALUE"])
df_panel["TURN_VOL"] = safe_log(df_panel["TURNOVER BY VOLUME"])
df_panel["LOG_PRICE"] = safe_log(df_panel["PRICE INDEX"])


<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>2.4</small>&nbsp;&nbsp;&nbsp;&nbsp;Interaction Terms:</h3>
</div>

In [2039]:
df_panel["SIZE_BM"]   = df_panel["SIZE"] * df_panel["BM"]
df_panel["SIZE_MOM"]  = df_panel["SIZE"] * df_panel["MOM12"]
df_panel["SIZE_VOL"]  = df_panel["SIZE"] * df_panel["VOL12"]
df_panel["SIZE_LIQ"]  = df_panel["SIZE"] * df_panel["TURN_VAL"]
df_panel["BM_MOM"] = df_panel["BM"] * df_panel["MOM12"]
df_panel["BM_VOL"] = df_panel["BM"] * df_panel["VOL12"]
df_panel["BM_LIQ"] = df_panel["BM"] * df_panel["TURN_VAL"]
df_panel["MOM_VOL"] = df_panel["MOM12"] * df_panel["VOL12"]
df_panel["MOM_LIQ"] = df_panel["MOM12"] * df_panel["TURN_VAL"]
df_panel["MOM_PRICE"] = df_panel["MOM12"] * df_panel["LOG_PRICE"]
df_panel["EY_MOM"] = df_panel["EY"] * df_panel["MOM12"]
df_panel["DY_SIZE"] = df_panel["DY"] * df_panel["SIZE"]

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>2.5</small>&nbsp;&nbsp;&nbsp;&nbsp;Standarization & Winsorization :</h3>
</div>

In [2042]:
characteristics = [
    "SIZE","BM","DY","EY","MOM3","MOM12","VOL12",
    "TURN_VAL","TURN_VOL","LOG_PRICE"
]

interaction_terms = [
    "SIZE_BM","SIZE_MOM","SIZE_VOL","SIZE_LIQ",
    "BM_MOM","BM_VOL","BM_LIQ",
    "MOM_VOL","MOM_LIQ","MOM_PRICE",
    "EY_MOM","DY_SIZE"
]

predictors = characteristics + interaction_terms

In [2043]:
df_panel[predictors] = (
    df_panel
    .groupby("Stock")[predictors]
    .shift(1)
)

In [2044]:
print(df_panel.head())

Variable       Date                Stock  DIVIDEND YIELD  MARKET VALUE  \
0        2005-01-01             3I GROUP            2.13       4085.77   
1        2005-01-01  A P MOLLER MAERSK B            0.66     100668.40   
2        2005-01-01                  A2A            2.94       3060.08   
3        2005-01-01             AALBERTS            1.54        878.29   
4        2005-01-01            ABB LTD N            0.00      13477.75   

Variable  MRKT VALUE TO BOOK   PER  PRICE INDEX        Rf  TOT RETURN IND  \
0                       1.12  30.4        244.9  0.001806          302.56   
1                       1.75   9.1       5126.9  0.001806         6348.85   
2                       1.86  15.7        199.7  0.001806          231.36   
3                       2.94  20.4       1378.8  0.001806         1844.24   
4                       2.93  10.4        292.8  0.001806          579.90   

Variable  TURNOVER BY VALUE  ...  SIZE_VOL  SIZE_LIQ  BM_MOM  BM_VOL  BM_LIQ  \
0           

In [2045]:
print(f'We have {len(predictors)} ({len(characteristics)} characteristics and {len(interaction_terms)} interaction terms)')

We have 22 (10 characteristics and 12 interaction terms)


But first, following the GKN

In [2047]:
class CSStandardizer:
    def __init__(self, lower=0.01, upper=0.99):
        self.lower = lower
        self.upper = upper
        self.q_low = None
        self.q_high = None
        self.mean = None
        self.std = None

    def fit(self, df, cols):
        self.q_low  = df[cols].quantile(self.lower)
        self.q_high = df[cols].quantile(self.upper)
        self.mean   = df[cols].mean()
        self.std    = df[cols].std()
        return self

    def transform(self, df, cols):
        x = df[cols].clip(self.q_low, self.q_high, axis=1)
        return (x - self.mean) / self.std

In [2048]:
df_panel[predictors].describe().loc[["mean","std"]].round(3)


Variable,SIZE,BM,DY,EY,MOM3,MOM12,VOL12,TURN_VAL,TURN_VOL,LOG_PRICE,...,SIZE_VOL,SIZE_LIQ,BM_MOM,BM_VOL,BM_LIQ,MOM_VOL,MOM_LIQ,MOM_PRICE,EY_MOM,DY_SIZE
mean,9.134,inf,0.029,0.073,0.019,0.109,0.081,13.057,9.729,6.589,...,0.730,121.880,-inf,inf,inf,0.010,1.397,0.773,0.003,0.268
std,1.509,NaN,0.028,0.112,0.138,0.361,0.046,1.752,1.958,1.784,...,0.388,34.192,NaN,NaN,NaN,0.129,4.664,2.341,0.063,0.246


In [2049]:
df_FINAL= df_panel

In [2050]:
df_FINAL.to_csv(path + "FINAL.csv", index=False)

<hr style="border:none;height:4px;background:#fff;margin:1em 0;">


<div style="text-align: align;">
  <h2> <small>3</small>&nbsp;&nbsp;&nbsp;&nbsp;Splitting:</h2>
</div>

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">

<div style="text-align: align;">
  <h3> <small>3.1</small>&nbsp;&nbsp;&nbsp;&nbsp;Test-Train Splitting:</h3>
</div>

In [2055]:
df_FINAL["Date"] = pd.to_datetime(df_FINAL["Date"])

In [2056]:
train = df_FINAL[df_FINAL["Date"] < "2019-01-01"].copy()
test  = df_FINAL[df_FINAL["Date"] >= "2019-01-01"].copy()

print(train["Date"].min(), train["Date"].max())
print(test["Date"].min(), test["Date"].max())

2005-01-01 00:00:00 2018-12-01 00:00:00
2019-01-01 00:00:00 2025-12-01 00:00:00


In [2057]:
y_train = train["EXCESS_RET_FWD"]
y_test  = test["EXCESS_RET_FWD"]

In [2058]:
X_train = train[predictors]
X_test  = test[predictors]

In [2059]:
X_train.shape, y_train.shape

((84217, 22), (84217,))

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>3.2</small>&nbsp;&nbsp;&nbsp;&nbsp; Exluding NaN for Returns & Imputation :</h3>
</div>

In [2062]:
train = train.dropna(subset=["EXCESS_RET_FWD"])
test  = test.dropna(subset=["EXCESS_RET_FWD"])

In [2063]:
def cs_median_impute(df, cols):
    df[cols] = (
        df
        .groupby("Date")[cols]
        .transform(lambda x: x.fillna(x.median()))
    )
    return df


In [2064]:
train = cs_median_impute(train, predictors)
test  = cs_median_impute(test, predictors)


In [2065]:
missing_by_date = (
    train
    .groupby("Date")[predictors]
    .apply(lambda x: x.isna().any().any())
)

missing_by_date.value_counts()


False    155
True      13
Name: count, dtype: int64

In [2066]:
valid_dates = missing_by_date[~missing_by_date].index

train = train[train["Date"].isin(valid_dates)].copy()


In [2067]:
train[predictors].isna().sum().sum(), train["EXCESS_RET_FWD"].isna().sum()


(0, 0)

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>3.3</small>&nbsp;&nbsp;&nbsp;&nbsp; Rolling Window Cross-Validation:</h3>
</div>

In [2070]:
class RollingPanelSplit:
    """
    Rolling time-based split for panel data (Date × Stock)

    Parameters
    ----------
    train_months : int
        Length of training window in months
    val_months : int or None
        Length of validation window in months (None = no validation)
    test_months : int
        Length of test window in months
    step_months : int
        Step size of the rolling window
    """

    def __init__(self,
                 train_months=120,
                 val_months=None,
                 test_months=12,
                 step_months=12):

        self.train_months = train_months
        self.val_months = val_months
        self.test_months = test_months
        self.step_months = step_months

    def split(self, df, date_col="Date"):
        """
        Returns a list of (train_idx, val_idx, test_idx)
        """

        df = df.sort_values(date_col).copy()
        dates = df[date_col].drop_duplicates().sort_values()

        splits = []

        start = dates.min() + relativedelta(months=self.train_months)

        while True:
            train_start = start - relativedelta(months=self.train_months)
            train_end = start

            if self.val_months is not None:
                val_end = train_end + relativedelta(months=self.val_months)
                test_end = val_end + relativedelta(months=self.test_months)
            else:
                val_end = None
                test_end = train_end + relativedelta(months=self.test_months)

            if test_end > dates.max():
                break

            train_idx = df[
                (df[date_col] >= train_start) &
                (df[date_col] < train_end)
            ].index

            if self.val_months is not None:
                val_idx = df[
                    (df[date_col] >= train_end) &
                    (df[date_col] < val_end)
                ].index
            else:
                val_idx = None

            test_idx = df[
                (df[date_col] >= (val_end if val_end else train_end)) &
                (df[date_col] < test_end)
            ].index

            splits.append((train_idx, val_idx, test_idx))

            start += relativedelta(months=self.step_months)

        return splits


<hr style="border:none;height:4px;background:#fff;margin:1em 0;">


<div style="text-align: align;">
  <h2> <small>4</small>&nbsp;&nbsp;&nbsp;&nbsp;Indicators Functions:</h2>
</div>

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>4.1</small>&nbsp;&nbsp;&nbsp;&nbsp; $R^2_{OOS}$ &nbsp;Function:</h3>
</div>

In [2075]:
def oos_r2(df, y_true="y_true", y_pred="y_pred", date_col="Date"):
    df = df.copy()
    df = df.sort_values(date_col)

    m = df.groupby(date_col)[y_true].mean()
    m_hist = m.expanding().mean().shift(1)

    df["mean_fc"] = df[date_col].map(m_hist)

    valid = (
        df["mean_fc"].notna()
        & df[y_true].notna()
        & df[y_pred].notna()
    )

    if valid.sum() == 0:
        return np.nan

    sse = ((df.loc[valid, y_true] - df.loc[valid, y_pred]) ** 2).sum()
    sst = ((df.loc[valid, y_true] - df.loc[valid, "mean_fc"]) ** 2).sum()

    if not np.isfinite(sst) or sst <= 0:
        return np.nan

    return 1 - sse / sst



<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>4.1</small>&nbsp;&nbsp;&nbsp;&nbsp; H-L Portifolio Function:</h3>
</div>

In [2078]:
def hl_month(df, q=0.1, pred_col="y_pred", ret_col="y_true"):
    """
    High-Low portfolio return for one cross-section
    """
    df = df[[pred_col, ret_col]].dropna()
    if df.empty:
        return np.nan

    q_low = df[pred_col].quantile(q)
    q_high = df[pred_col].quantile(1 - q)

    return (
        df.loc[df[pred_col] >= q_high, ret_col].mean()
        - df.loc[df[pred_col] <= q_low, ret_col].mean()
    )



<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>4.1</small>&nbsp;&nbsp;&nbsp;&nbsp; $Sharpe$ (H-L) Function:</h3>
</div>

In [2081]:
def sharpe(returns, periods_per_year=12):
    mean = returns.mean()
    std  = returns.std()
    return np.sqrt(periods_per_year) * mean / std



In [2082]:
def sharpe_gkn(returns, oos_r2, periods_per_year=12):
    SR = sharpe(returns, periods_per_year)
    return np.sqrt((SR**2 + oos_r2) / (1 - oos_r2))


In [2083]:
splitter = RollingPanelSplit(
    train_months=120,   
    val_months=None,    
    test_months=12,    
    step_months=12      
)
splits = splitter.split(df_FINAL, date_col="Date")


In [2084]:
def run_rolling_ols(df, splits, predictors, target):
    """
    Rolling OOS OLS with proper lagging and train-only standardization
    """
    oos_results = []

    for split_id, (train_idx, _, test_idx) in enumerate(splits):

        train = df.loc[train_idx].copy()
        test  = df.loc[test_idx].copy()

        train = train.dropna(subset=predictors + [target])
        test  = test.dropna(subset=predictors + [target])

        if len(train) == 0 or len(test) == 0:
            continue

        # ----------------------------
        # Winsorazation
        # ----------------------------
        stdzr = CSStandardizer(lower=0.01, upper=0.99)
        stdzr.fit(train, predictors)

        X_train = stdzr.transform(train, predictors).values
        X_test  = stdzr.transform(test, predictors).values

        y_train = train[target].values
        y_test  = test[target].values

        # ----------------------------
        # OLS FULL
        # ----------------------------
        model = LinearRegression(fit_intercept=True)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        out = test[["Date", "Stock"]].copy()
        out["y_true"] = y_test
        out["y_pred"] = y_pred
        out["model"]  = "OLS"
        out["split"]  = split_id

        oos_results.append(out)

    return pd.concat(oos_results, ignore_index=True)


In [2085]:
target = "EXCESS_RET_FWD"


In [2086]:
ols_full = run_rolling_ols(
    df=df_FINAL,
    splits=splits,
    predictors=predictors,
    target=target
)


ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
ols_full = pd.concat(oos_results, ignore_index=True)


In [ ]:
ols_full.columns


In [ ]:
r2_ols = oos_r2(ols_full)
print(f"OOS R² (OLS): {r2_ols:.4f}")


In [ ]:
hl_ols_full = (
    ols_full
    .groupby("Date")[["y_pred", "y_true"]]
    .apply(hl_month)
)

In [ ]:
type(hl_ols_monthly), hl_ols_monthly.shape
hl_ols_monthly.head()

In [ ]:
sharpe_ols_full = sharpe_gkn(hl_ols_monthly, r2_ols,periods_per_year=12)

print(f"Annualized Sharpe ratio (OLS H–L): {sharpe_ols_full:.2f}")


In [ ]:
ols_full.groupby("Date")["y_pred"].std().describe()


In [ ]:
tmp = ols_full.copy()
tmp["y_pred"] = np.random.randn(len(tmp))

hl_placebo = (
    tmp.groupby("Date")[["y_pred", "y_true"]]
       .apply(hl_month)
)

print("Placebo Sharpe:", sharpe(hl_placebo))


In [ ]:
for q in [0.1, 0.2, 0.3, 0.4]:
    hl_q = (
        ols_full.groupby("Date")[["y_pred", "y_true"]]
               .apply(lambda m: hl_month(m, q=q))
    )
    print(q, sharpe(hl_q))


In [ ]:
ols_test = ols_full.copy()
ols_test["y_pred"] = ols_test.groupby("Stock")["y_pred"].shift(1)

hl_shifted = (
    ols_test
    .groupby("Date")[["y_pred", "y_true"]]
    .apply(hl_month)
)

print("Shifted Sharpe:", sharpe(hl_shifted))
